In [76]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [77]:
# Load the datasets
original_sales_data = pd.read_csv('../data/snp_dld_2024_transactions.csv', low_memory=False)
original_rentals_data = pd.read_csv('../data/snp_dld_2024_rents.csv', low_memory=False)

In [78]:
columns_to_drop_sales = [
    'transaction_number',  # ID column
    'entry_id',            # Metadata
    'meta_ts',             # Metadata timestamp
    'master_project_en',   # Very few non-null values
    'master_project_ar',   # Very few non-null values
    'property_type_ar',    # Duplicate of property_type_en
    'property_subtype_ar', # Duplicate of property_subtype_en
    'rooms_ar',            # Duplicate of rooms_en
    'project_name_ar',     # Duplicate of project_name_en
    'area_ar',             # Duplicate of area_en
    'nearest_landmark_ar', # Duplicate of nearest_landmark_en
    'nearest_metro_ar',    # Duplicate of nearest_metro_en
    'nearest_mall_ar',      # Duplicate of nearest_mall_en
    'parcel_id',  # not needed
    'transaction_type_en', #Duplicate of transaction_type_id
    'transaction_subtype_en', #Constant
    'transaction_subtype_id', #Constant
    'property_id', #Constant
    'property_type_id', #Constant
    'property_subtype_id', #Constant
    'building_age', #Constant
    'area_id', #Constant
    'is_freehold_text', #duplicate of is_freehold
    'property_usage_en', # Duplicate of property_usage_id
    'transaction_datetime_year',  #Constant after handling dates
    'req_from_year',  #Constant after handling dates
    'req_from_day',  #Constant after handling dates
    'req_to_year', #Constant after handling dates
    'parking' # Replaced with parking count
]

# List of columns to drop from rentals_data
columns_to_drop_rentals = [
    'ejari_contract_number', # ID column
    'land_property_id',      # Redundant ID column
    'entry_id',              # Metadata
    'meta_ts',               # Metadata timestamp
    'master_project_en',     # Very few non-null values
    'master_project_ar',     # Very few non-null values
    'property_type_ar',      # Duplicate of property_type_en
    'property_subtype_ar',   # Duplicate of property_subtype_en
    'property_usage_ar',     # Duplicate of property_usage_en
    'project_name_ar',       # Duplicate of project_name_en
    'area_ar',               # Duplicate of area_en
    'nearest_landmark_ar',   # Duplicate of nearest_landmark_en
    'nearest_metro_ar',      # Duplicate of nearest_metro_en
    'nearest_mall_ar',        # Duplicate of nearest_mall_en
    'property_id',   #constant
    'property_usage_id', #constant
    'area_id',   #constant
    'ejari_property_type_id',   #constant
    'ejari_property_sub_type_id',  #constant
    'parking', #97% missing data
    'parcel_id', # negative correlation
    'registration_date_year',  #Constant after handling dates
    'req_from_year',  #Constant after handling dates
    'req_from_day',  #Constant after handling dates
    'req_to_year',  #Constant after handling dates
    'req_from_year',  #Constant after handling dates
    'req_from_day', #Constant after handling dates
    'req_to_year',  #Constant after handling dates
    'registration_date_year',  #Constant after handling dates
    'req_from_year',  #Constant after handling dates
    'req_from_day',  #Constant after handling dates
    'req_to_year', #Constant after handling dates
    'is_freehold_text' # duplicate of is_freehold
]

Handle Dates

In [79]:
def process_date_columns(df, date_columns):
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
            df[f"{col}_year"] = df[col].dt.year
            df[f"{col}_month"] = df[col].dt.month
            df[f"{col}_day"] = df[col].dt.day
            df[f"{col}_weekday"] = df[col].dt.weekday
            df[f"{col}_dayofyear"] = df[col].dt.dayofyear
    df.drop(columns=date_columns, inplace=True, errors='ignore')
    return df

sales_date_columns = ['transaction_datetime', 'req_from', 'req_to']
rentals_date_columns = ['registration_date', 'contract_start_date', 'contract_end_date', 'req_from', 'req_to']
sales_data = process_date_columns(original_sales_data, sales_date_columns)
rentals_data = process_date_columns(original_rentals_data, rentals_date_columns)

# Handle Parking in Sales values

In [80]:
import re
import pandas as pd

def calculate_parking_count(value):
    """
    Calculate the parking count based on the given strategy:
    - Fields separated by commas are counted as separate spots unless 'to' is present.
    - If 'to' is present, calculate the range of parking spots.
    - Single identifiers like 'G-127' are counted as one spot.
    """
    if pd.isna(value):
        return 0  # No information

    total_count = 0
    fields = str(value).split(",")  # Split by commas
    for field in fields:
        field = field.strip()  # Remove extra whitespace
        if re.search(r"\bto\b", field, re.IGNORECASE):  # Check for 'to' (case insensitive)
            # Extract numbers in the range and calculate spots
            numbers = [int(num) for num in re.findall(r"\d+", field)]
            if len(numbers) == 2:  # Range is valid
                total_count += abs(numbers[1] - numbers[0]) + 1
        elif re.search(r"\d+", field):  # Single numeric identifier (e.g., G-127)
            total_count += 1
        else:
            total_count += 0  # Invalid or non-informative field
    return total_count

# Apply the logic to the `parking` column
sales_data['parking_count'] = sales_data['parking'].apply(calculate_parking_count)

# Save results for validation
sales_data[['parking', 'parking_count']].to_csv("processed_parking_count.csv", index=False)

# Example check
print(sales_data[['parking', 'parking_count']].tail())

       parking  parking_count
162801  B2-081              1
162802     NaN              0
162803  B2-140              1
162804   G-114              1
162805   G-127              1


# Drop columns

In [81]:
sales_data.drop(columns=columns_to_drop_sales, inplace=True)
rentals_data.drop(columns=columns_to_drop_rentals, inplace=True)


In [83]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162806 entries, 0 to 162805
Data columns (total 30 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   transaction_type_id             162806 non-null  int64  
 1   registration_type_en            162806 non-null  object 
 2   property_usage_id               162806 non-null  int64  
 3   amount                          162806 non-null  float64
 4   total_buyer                     162806 non-null  int64  
 5   total_seller                    162806 non-null  int64  
 6   transaction_size_sqm            162146 non-null  float64
 7   property_size_sqm               162806 non-null  float64
 8   is_offplan                      162806 non-null  object 
 9   is_freehold                     162122 non-null  object 
 10  property_type_en                162806 non-null  object 
 11  property_subtype_en             161414 non-null  object 
 12  rooms_en        

In [23]:
rentals_data['version_number'].unique()

array([12, 14, 13, 11, 10,  5,  9,  7,  8,  4,  6,  1,  3, 15,  2, 16, 17,
       18, 21, 22, 23, 26, 19, 27, 24])

In [25]:
rentals_data['version_text'].unique()

array(['Renewed', 'New'], dtype=object)

In [40]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162806 entries, 0 to 162805
Data columns (total 31 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   transaction_type_id             162806 non-null  int64  
 1   transaction_subtype_id          162806 non-null  int64  
 2   registration_type_en            162806 non-null  object 
 3   property_usage_id               162806 non-null  int64  
 4   amount                          162806 non-null  float64
 5   total_buyer                     162806 non-null  int64  
 6   total_seller                    162806 non-null  int64  
 7   transaction_size_sqm            162146 non-null  float64
 8   property_size_sqm               162806 non-null  float64
 9   is_offplan                      162806 non-null  object 
 10  is_freehold                     162122 non-null  object 
 11  property_type_en                162806 non-null  object 
 12  property_subtype

In [51]:
def print_unique_values(df):
    """
    Iterate through all columns in a DataFrame.
    Skip columns of type float, and print unique values for other column types.

    Parameters:
        df (pd.DataFrame): The DataFrame to analyze.
    """
    for col in df.columns:
        if df[col].dtype == 'float64':
            continue  # Skip columns with float type
        print(f"Unique values in '{col}':")
        print(df[col].unique())
        print("\n")

In [52]:
print_unique_values(sales_data)

Unique values in 'transaction_type_id':
[1 2 3]


Unique values in 'transaction_subtype_id':
[0]


Unique values in 'registration_type_en':
['Off-Plan' 'Ready']


Unique values in 'property_usage_id':
[1 2]


Unique values in 'total_buyer':
[ 2  1  3  0  4  5  8  6  7 21  9 10 12]


Unique values in 'total_seller':
[ 1  2  0  4 11  3  6  7  5  8  9 16 23 14 12 17 10 15 18 28 13 24 19 21
 30 26 48 22 20 31]


Unique values in 'is_offplan':
['t' 'f']


Unique values in 'is_freehold':
['t' 'f' nan]


Unique values in 'property_type_en':
['Unit' 'Building' 'Land']


Unique values in 'property_subtype_en':
['Flat' 'Villa' 'Hotel Apartment' 'Hotel Rooms' 'Government Housing'
 'Residential' 'Shop' 'Office' nan 'Commercial' 'Industrial' 'Land'
 'Residential Flats' 'Hotel' 'Airport' 'Stacked Townhouses' 'Unit'
 'Sports Club' 'School' 'Agricultural' 'General Use' 'Labor Camp' 'Clinic'
 'Building' 'Petrol Station' 'Commercial / Offices / Residential'
 'Show Rooms' 'Workshop' 'Electricity Station'

In [55]:
sales_data['parking'].describe()

count     122343
unique     18174
top            1
freq       59218
Name: parking, dtype: object

In [57]:
print("Missing values in 'parking':", sales_data['parking'].isnull().sum())


Missing values in 'parking': 40463


In [58]:
sales_data['parking'].unique()

array(['1', '1  0', '2', ..., 'G257', 'P3-16,17,18', 'G-328,G-329'],
      dtype=object)

In [59]:
# Get the unique values of the 'parking' column
unique_parking_values = sales_data['parking'].unique()

# Convert unique values to a list (optional)
unique_parking_list = unique_parking_values.tolist()

# Save the unique values to a text file
with open("unique_parking_values.txt", "w") as file:
    for value in unique_parking_list:
        file.write(f"{value}\n")

print("Unique values saved to 'unique_parking_values.txt'")


Unique values saved to 'unique_parking_values.txt'


In [60]:
import pandas as pd
import numpy as np
import re

def preprocess_parking_column(df, column_name):
    # Extract unique values
    unique_values = df[column_name].unique()
    
    # Categorize values
    def categorize_parking(value):
        if pd.isnull(value):
            return 'Unknown'
        elif re.match(r'^\d+$', str(value)):  # Pure numerical values
            return 'Count'
        elif re.match(r'[A-Z]-\d+', str(value)):  # Codes like B1-19
            return 'Parking Code'
        elif re.match(r'.*\d+.*', str(value)):  # Contains numbers in text
            return 'Descriptive with Numbers'
        else:  # Default for others
            return 'Other'
    
    # Apply categorization
    df[f'{column_name}_category'] = df[column_name].apply(categorize_parking)
    
    # Handle known categories separately if necessary
    df[f'{column_name}_processed'] = df[column_name].fillna('Unknown')
    
    # Example: Encode numerical counts as-is
    df[f'{column_name}_processed'] = df.apply(
        lambda x: x[column_name] if x[f'{column_name}_category'] == 'Count' else x[f'{column_name}_processed'],
        axis=1
    )
    
    return df

# Example usage
sales_data = preprocess_parking_column(sales_data, 'parking')

# Check the distribution
print(sales_data['parking_category'].value_counts())


parking_category
Count                       64893
Unknown                     40463
Descriptive with Numbers    38334
Parking Code                17468
Other                        1648
Name: count, dtype: int64


In [62]:
sales_data['parking_category'].unique()

array(['Count', 'Descriptive with Numbers', 'Parking Code', 'Unknown',
       'Other'], dtype=object)

In [68]:
import re

# Extract parking counts from descriptions
def extract_parking_count(value):
    match = re.search(r'\b\d+\b', str(value))
    return int(match.group()) if match else None

# Categorize parking descriptions
def categorize_parking_description(value):
    if "limited" in str(value).lower():
        return "Limited"
    elif "available" in str(value).lower():
        return "Available"
    else:
        return "Other"

# Process parking feature
sales_data['parking_count'] = sales_data['parking'].apply(extract_parking_count)
sales_data['parking_category'] = sales_data['parking'].apply(categorize_parking_description)

# Handle 'Unknown'
sales_data['parking_count'] = sales_data['parking_count'].fillna(
    sales_data.groupby('area_en')['parking_count'].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 0)
    )
)


# Validate processed data
print(sales_data[['parking_count', 'parking_category']])


        parking_count parking_category
0                 1.0            Other
1                 1.0            Other
2                 1.0            Other
3                 1.0            Other
4                 1.0            Other
...               ...              ...
162801           81.0            Other
162802            0.0            Other
162803          140.0            Other
162804          114.0            Other
162805          127.0            Other

[162806 rows x 2 columns]


In [69]:
import re

# Function to extract parking count
def extract_parking_count(value):
    match = re.search(r'\b\d+\b', str(value))  # Find standalone numbers
    return int(match.group()) if match else 0  # Default to 0 if no match

# Function to categorize parking description
def categorize_parking(value):
    if "limited" in str(value).lower():
        return "Limited"
    elif "available" in str(value).lower():
        return "Available"
    elif re.search(r'[A-Za-z]*\d+[A-Za-z]*', str(value)):  # Matches codes like B1-19
        return "Code-Based"
    else:
        return "Unknown"

# Create new columns
sales_data['parking_available'] = sales_data['parking'].apply(
    lambda x: 1 if x != "Unknown" and pd.notnull(x) else 0
)
sales_data['parking_count'] = sales_data['parking'].apply(extract_parking_count)
sales_data['parking_category'] = sales_data['parking'].apply(categorize_parking)

# Validate the results
print(sales_data[['parking', 'parking_available', 'parking_count', 'parking_category']].head())


  parking  parking_available  parking_count parking_category
0       1                  1              1       Code-Based
1       1                  1              1       Code-Based
2       1                  1              1       Code-Based
3       1                  1              1       Code-Based
4       1                  1              1       Code-Based


In [71]:
print(sales_data[['parking', 'parking_available', 'parking_count', 'parking_category']].tail())


       parking  parking_available  parking_count parking_category
162801  B2-081                  1             81       Code-Based
162802     NaN                  0              0          Unknown
162803  B2-140                  1            140       Code-Based
162804   G-114                  1            114       Code-Based
162805   G-127                  1            127       Code-Based


In [75]:

import re
import pandas as pd

def calculate_parking_count(value):
    """
    Calculate the parking count based on the given strategy:
    - Fields separated by commas are counted as separate spots unless 'to' is present.
    - If 'to' is present, calculate the range of parking spots.
    - Single identifiers like 'G-127' are counted as one spot.
    """
    if pd.isna(value):
        return 0  # No information

    total_count = 0
    fields = str(value).split(",")  # Split by commas
    for field in fields:
        field = field.strip()  # Remove extra whitespace
        if re.search(r"\bto\b", field, re.IGNORECASE):  # Check for 'to' (case insensitive)
            # Extract numbers in the range and calculate spots
            numbers = [int(num) for num in re.findall(r"\d+", field)]
            if len(numbers) == 2:  # Range is valid
                total_count += abs(numbers[1] - numbers[0]) + 1
        elif re.search(r"\d+", field):  # Single numeric identifier (e.g., G-127)
            total_count += 1
        else:
            total_count += 0  # Invalid or non-informative field
    return total_count

# Apply the logic to the `parking` column
sales_data['parking_count'] = sales_data['parking'].apply(calculate_parking_count)

# Save results for validation
sales_data[['parking', 'parking_count']].to_csv("processed_parking_count.csv", index=False)

# Example check
print(sales_data[['parking', 'parking_count']].tail())


       parking  parking_count
162801  B2-081              1
162802     NaN              0
162803  B2-140              1
162804   G-114              1
162805   G-127              1


In [74]:
sales_data[["parking", "parking_category", "parking_count", "parking_available"]].tail()

,parking,parking_category,parking_count,parking_available
162801,B2-081,Descriptive with Numbers,NaN,1
162802,NaN,Unknown,0.0,0
162803,B2-140,Descriptive with Numbers,NaN,1
162804,G-114,Descriptive with Numbers,NaN,1
162805,G-127,Descriptive with Numbers,NaN,1


In [ ]:
sales_categorical_values = [
    'transaction_type_id',
    'registration_type_en',
    'property_usage_id',
    
]

In [45]:
sales_data['transaction_type_id'].unique()

array([1, 2, 3])

In [48]:
original_sales_data['transaction_subtype_id'].unique()

array([0])

In [49]:
sales_data['registration_type_en'].unique()

array(['Off-Plan', 'Ready'], dtype=object)

In [50]:
sales_data['property_usage_id'].unique()

array([1, 2])

# Handling Missing Values

In [84]:
# Display the number and percentage of missing values for Sales Data
print("Missing Values in Sales Data:")
missing_sales = sales_data.isnull().sum().to_frame(name="Missing Count")
missing_sales["Percentage"] = (missing_sales["Missing Count"] / len(sales_data)) * 100
print(missing_sales[missing_sales["Missing Count"] > 0])

# Display the number and percentage of missing values for Rentals Data
print("\nMissing Values in Rentals Data:")
missing_rentals = rentals_data.isnull().sum().to_frame(name="Missing Count")
missing_rentals["Percentage"] = (missing_rentals["Missing Count"] / len(rentals_data)) * 100
print(missing_rentals[missing_rentals["Missing Count"] > 0])

Missing Values in Sales Data:
                      Missing Count  Percentage
transaction_size_sqm            660    0.405390
is_freehold                     684    0.420132
property_subtype_en            1392    0.855005
rooms_en                      23257   14.285100
project_name_en               24760   15.208285
nearest_landmark_en           47412   29.121777
nearest_metro_en              65982   40.527990
nearest_mall_en               66842   41.056226

Missing Values in Rentals Data:
                     Missing Count  Percentage
annual_amount                   14    0.001873
property_subtype_en           2545    0.340510
property_usage_en             2844    0.380515
rooms                       721746   96.566534
project_name_en             576935   77.191440
nearest_landmark_en          56810    7.600935
nearest_metro_en            110681   14.808645
nearest_mall_en             120826   16.166003
